In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = input("Please paste your handshake url")

connection_attempts = 3

while connection_attempts > 0:
    response = requests.get(url)

    if response.status_code == 200:
        print("Successfully connected")
        soup = BeautifulSoup(response.text, "html")
        print(soup)
        break

    elif response.status_code == 429:
         print("Response [429], Reattempting to connect")
         connection_attempts -= 1

    elif response.status_code == 403:
            print("Access denied") 
    else:
         print("Invalid link")




In [ ]:
# Find and print all tags and their contents
for tag in soup.find_all(True):
    print(f"Tag: <{tag.name}>")
    print(f"Contents: {tag.prettify()}")
    print("-" * 40)


In [38]:
soup.find_all("a")

[<a class="logo" href="https://www.joinhandshake.com"><img alt="Handshake logo image" height="24" src="https://handshake-production-cdn.joinhandshake.com/assets/logo-icon-42378cd841a06f65f897c87fcb32c4c27e6e4b686fb7e65a9420e9a0605386db.svg"/>
 </a>,
 <a data-track-click="login-v2-clicked-signup-app-domain" href="/student_account_registrations/new">Sign up here.</a>,
 <a data-track-click="login-v2-clicked-signup-app-domain" href="/student_account_registrations/new">Sign up here.</a>,
 <a class="sso-button" data-bind="click: track_sso_click" href="https://sjsu.okta.com/app/sanjosestateuniversity_sjsuhandshake_1/exk5536is1LPSizqM0x7/sso/saml?SAMLRequest=hZJLb8IwEIT%2FSm4%2B5WXCywqRKByKBCqCtIdekAmbYkjskHWqtL%2B%2BDq%2FmUnreb2Z27QmR51nBxpXeyxWcKkBtjRGh1ELJiZJY5VCuofwUCbyu5iOy17pA5rq8KJyDEnLP5Q73%2FAhOonK3cdskFxmxpsZNSN5Y%2FQrxgJWjjpqfBcbGiORBoWG5hkqKTyhR6K9NA97dN74L9bHb7fQE%2BvPlWnyfFl7ddxHVOZRYs%2BmIbHrd1OuAD3aPU24HfDC0%2BTbt2x0%2FGAyHFAZ9SA2KWMFMmkCpR4R6NLC9gU1pTD1GA%2BYH78RalkqrRGVPQu6E%